# Required Libraries

In [ ]:
%pip install PyPDF2 tesseract opencv-python pytesseract pandas
!sudo apt install tesseract-ocr
!sudo apt-get install poppler-utils
!pip install pdf2image
!pip install transformers
!pip install pyspellchecker
!pip install symspellpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562572 sha256=38d9fc87195df7c6b5f001cb82ecfff7a3bc7ac37b3e8de95551a3dc3b0e842f
  Stored in directory: /root/.cache/pip/wheels/6c/c5/81/8310cc52076953e53412ed1875a5e224c92940235bdcee21a2
Successfully built tesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 23 not upgraded.
Need to get 4,850 kB of archives.
After this operation

In [ ]:
!pip install camelot-py[cv]
!pip install tabula-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 95.2 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import pytesseract
import cv2
import pandas as pd
import numpy as np
import io
from PIL import Image
from pdf2image import convert_from_bytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/NoName.Co/academic/example_pdf

/content/drive/MyDrive/NoName.Co/academic/example_pdf


In [ ]:
ls

example_pdf/  fine-tuning-image-generator.ipynb  pdf_regions.ipynb


# Text Extraction

## Directly from PDF

In [ ]:
import PyPDF2

# Open the PDF file in read-binary mode
with open('example.pdf', 'rb') as pdf_file:

    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Get the total number of pages in the PDF file
    num_pages = len(pdf_reader.pages)

    # Extract text from each page of the PDF file
    text = ''
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        page_text = page.extract_text()
        text += page_text

# Print the extracted text
print(text)

ModuleNotFoundError: ignored

## Converting to image and then extracting text

In [ ]:
# Define a function to extract text from a PDF file in memory
def extract_text_from_pdf(pdf_bytes):
    # Convert the PDF file to a list of PIL images using pdf2image
    images = convert_from_bytes(pdf_bytes)

    # Extract text from each image using Tesseract
    text = ''
    for image in images:
        # Convert the image to grayscale
        image = image.convert('L')
        
        # Extract text from the image using Tesseract
        page_text = pytesseract.image_to_string(image, lang='eng')
        text += page_text
    
    return text

# Example usage: read a PDF file from a local file and extract the text
with open('example.pdf', 'rb') as pdf_file:
    pdf_bytes = pdf_file.read()
    text = extract_text_from_pdf(pdf_bytes)
    print(text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
3612

IEEE TRANSACTIONS ON MICROWAVE THEORY AND TECHNIQUES, VOL. 64, NO. 11, NOVEMBER 2016

Voltage-Combined CMOS Doherty Power
Amplifier Based on Transformer

Yunsung Cho, Member, IEEE, Kyunghoon Moon, Student Member, IEEE,
Byungjoon Park, Member, IEEE, Jooseung Kim, Member, IEEE,
and Bumman Kim, Fellow, IEEE

Abstract—This paper presents a voltage-combined CMOS
Doherty power amplifier (PA) based on a transformer. The
dynamic load modulation of the two-way transformer is analyzed
in detail. A comparison between the current- and voltage-
combined Doherty PAs shows that the Doherty PA based on
a voltage-combining method delivers a broad bandwidth (BW)
with a larger load tolerance. The proposed Doherty PA achieves
close to ideal load modulation without additional offset line
by manipulation of the output capacitances. For demonstration
purposes, the PA and output combining transformer are i

## Text without image regions

In [ ]:
!pip install pdf2image pytesseract opencv-python-headless numpy

import io
import cv2
import numpy as np
from pdf2image import convert_from_bytes
from PIL import Image
import pytesseract

# Define a function to extract text from a PDF file in memory, excluding images and charts
def extract_text_from_pdf(pdf_bytes):
    # Convert the PDF file to a list of PIL images using pdf2image
    images = convert_from_bytes(pdf_bytes)

    # Extract text from each image using Tesseract
    text = ''
    for image in images:
        # Convert the PIL image to a NumPy array
        image_np = np.array(image)

        # Remove image and chart regions using OpenCV
        gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        mask = np.ones(image_np.shape[:2], dtype="uint8") * 255
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 1000:
                cv2.drawContours(mask, [contour], -1, 0, -1)
        image_np[mask == 0] = [255, 255, 255]

        # Convert the NumPy array back to a PIL image
        image = Image.fromarray(image_np)

        # Extract text from the image using Tesseract
        page_text = pytesseract.image_to_string(image, lang='eng')
        text += page_text
    
    return text

# Example usage: read a PDF file from a local file and extract the text
with open('example.pdf', 'rb') as pdf_file:
    pdf_bytes = pdf_file.read()
    text = extract_text_from_pdf(pdf_bytes)
    print(text)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
3612

Voltage-Combined C
ased on Transformer

Amplifier

IEEE TRANSACTIONS ON MICROWAVE THEORY AND TECHNIQUES, VOL. 64, NO. 11, NOVEMBER 2016

S oherty Power

Yunsung Cho, Member, IEEE, Kyunghoon Moon, Student Member, IEEE,
Byungjoon Park, Member, IEEE, Jooseung Kim, Member, IEEE,
and Bumman Kim, Fellow, IEEE

Abstract—This paper presents a voltage-combined CMOS
Doherty power amplifier (PA) based on a transformer. The
dynamic load modulation of the two-way transformer is analyzed
in detail. A comparison between the current- and voltage-
combined Doherty PAs shows that the Doherty PA based on
a voltage-combining method delivers a broad bandwidth (BW)
with a larger load tolerance. The proposed Doherty PA achieves
close to ideal load modulation without additional offset line
by manipulation of the output capacitances. For demonstration
purposes, the PA and output combining transformer are im

## Text Cleaner

In [ ]:
import re
from symspellpy import SymSpell, Verbosity
import requests

def clean_text(text: str) -> str:
    # Remove URLs
    text = re.sub(r'http[s]?://\S+', '', text)

    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)

    # Remove special LaTeX characters
    text = re.sub(r'\\[a-zA-Z]+', '', text)

    # Remove multiple whitespaces
    text = re.sub(r'\s+', ' ', text)

    # Remove non-ASCII characters
    text = text.encode("ascii", "ignore").decode()

    # Remove unwanted symbols
    unwanted_symbols = ['{', '}', '^', '_', '*', '+', '/', '<', '=', '>', '@', '|', '~']
    for symbol in unwanted_symbols:
        text = text.replace(symbol, '')

    return text.strip()

def correct_spelling(text: str) -> str:
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
    word_list = requests.get("https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt").text.splitlines()
    sym_spell.create_dictionary(word_list)

    words = text.split()
    corrected_words = []
    
    for word in words:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_words.append(suggestions[0].term)
        else:
            corrected_words.append(word)
    
    return ' '.join(corrected_words)



In [ ]:
# Example usage
extracted_text = text
cleaned_text = clean_text(extracted_text)
corrected_text = correct_spelling(cleaned_text)

print(f"Corrected text: {corrected_text}")


In [ ]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering

tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

def split_document(document: str, max_length: int) -> list:
    words = document.split(" ")
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        if current_length + len(word) + 1 <= max_length:
            current_chunk.append(word)
            current_length += len(word) + 1
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = len(word) + 1
    chunks.append(" ".join(current_chunk))
    return chunks

def extract_answer(document: str, question: str) -> str:
    max_length = 512 - tokenizer.num_special_tokens_to_add(pair=True)
    document_chunks = split_document(document, max_length)

    best_score = -float("inf")
    best_start, best_end = 0, 0
    best_chunk_id = 0

    for i, chunk in enumerate(document_chunks):
        inputs = tokenizer(question, chunk, return_tensors="pt", max_length=512, truncation=True)
        input_ids = inputs["input_ids"].tolist()[0]

        outputs = model(**inputs)
        answer_start_scores, answer_end_scores = outputs.start_logits, outputs.end_logits

        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1

        if answer_start_scores[0][answer_start].item() > best_score:
            best_score = answer_start_scores[0][answer_start].item()
            best_start, best_end = answer_start, answer_end
            best_chunk_id = i

    input_ids = tokenizer(question, document_chunks[best_chunk_id], return_tensors="pt", max_length=512, truncation=True)["input_ids"].tolist()[0]
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[best_start:best_end]))
    return answer.strip()




In [ ]:
document = corrected_text
question = "What is the total gate width of the thin and thick devices?"

answer = extract_answer(document, question)
print(f"Answer: {answer}")

Answer: 4800 and 9600 wm


# Ready Example

In [ ]:
doc_names = ["example.pdf","example2.pdf","example3.pdf","example4.pdf"]
questions = ["Which technology is used for the research?",
             "How is the PA is implemented?",
             "How is the PA is fabricated?"
             "What is the gain?",
             "What is the overall efficiency?",
             "How is the PA tested?"]

df_text = pd.DataFrame(columns=['name', 'text', 'answers' ])


for doc in doc_names:
  with open(doc, 'rb') as pdf_file:
      pdf_bytes = pdf_file.read()
      text = extract_text_from_pdf(pdf_bytes)
      print(doc, "is read.")
      #print(text)

      cleaned_text = clean_text(text)
      corrected_text = correct_spelling(cleaned_text)
      print(doc, "is corrected.")

      list_of_ans=[]
      
      for q in questions:

        answer = extract_answer(corrected_text, q)
        list_of_ans.append(answer)
        #print(f"Answer: {answer}")

      df_text = df_text.append({
          'name': doc,
          'text': corrected_text,
          'answers': list_of_ans
          }, ignore_index=True)
      
      print(doc, "is answered.")


example.pdf is read.
example.pdf is corrected.


<ipython-input-8-fa3e5becfc54>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_text = df_text.append({


example.pdf is answered.
example2.pdf is read.
example2.pdf is corrected.


<ipython-input-8-fa3e5becfc54>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_text = df_text.append({


example2.pdf is answered.
example3.pdf is read.
example3.pdf is corrected.


<ipython-input-8-fa3e5becfc54>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_text = df_text.append({


example3.pdf is answered.
example4.pdf is read.
example4.pdf is corrected.
example4.pdf is answered.


<ipython-input-8-fa3e5becfc54>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_text = df_text.append({


In [ ]:
for i in range(len(df_text)):
  print(df_text['answers'].values[i])

['cmos', 'a proposed cmos linear doherty a a design of proposed boltage cmos cherty a', '13 . 3 d', '47 . 4 %', 'comparison between current - and voltage - combined doherty a big a']
['high - performance transistor', 'linearity polar modulation', 'high efficiency', 'po out pot out a a a 14', 'linearity polar modulation']
['cmos cherty a incor borating a bias boosting circuit', 'power amplifier', '27 . 2 d', '43 . 6 %', 'gate - voltage boosting']
['microwave', 'a driver stage', '1 . 17 d', '1 . 9 - ghz', 'compare different power - combining topologies']


# Pipeline

In [ ]:
doc_names = []
questions = []

df_text = pd.DataFrame(columns=['name', 'text', 'answers' ])


for doc in doc_names:
  with open(doc, 'rb') as pdf_file:
      pdf_bytes = pdf_file.read()
      text = extract_text_from_pdf(pdf_bytes)
      #print(text)

      cleaned_text = clean_text(text)
      corrected_text = correct_spelling(cleaned_text)

      list_of_ans=[]
      
      for q in questions:

        answer = extract_answer(corrected_text, q)
        list_of_ans.append(answer)
        #print(f"Answer: {answer}")

      df_text = df_text.append({
          'name': doc,
          'text': corrected_text,
          'answers': list_of_ans
          }, ignore_index=True)

  


# Table Recognition

In [ ]:
import camelot
import PyPDF2
import pandas as pd
from transformers import TapasTokenizer, TapasForQuestionAnswering

# Read PDF file
pdf_path = "example.pdf"
pdf_file = open(pdf_path, 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

# Extract tables using Camelot
tables = camelot.read_pdf(pdf_path, flavor='stream', pages='all')

# Process and store tables as Pandas DataFrames
data_frames = []
for table in tables:
    data_frame = table.df
    # Clean and preprocess the data_frame here as needed
    data_frames.append(data_frame)

pdf_file.close()

# Load pre-trained TAPAS model and tokenizer
model_name = "google/tapas-base-finetuned-wtq"
tokenizer = TapasTokenizer.from_pretrained(model_name)
model = TapasForQuestionAnswering.from_pretrained(model_name)

# Example question
question = "What is the value in the second row and third column?"

# Define a function to get the answer from a DataFrame
def get_answer(question, data_frame):
    inputs = tokenizer(table=data_frame, queries=[question], padding='max_length', return_tensors="pt")
    outputs = model(**inputs)
    predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
        inputs,
        outputs.logits.detach(),
        outputs.logits_aggregation.detach()
    )

    # Retrieve answer from coordinates
    answers = []
    for coordinates in predicted_answer_coordinates:
        if len(coordinates) == 1:
            answers.append(data_frame.iat[coordinates[0]])
        else:
            answers.append(", ".join([data_frame.iat[coord] for coord in coordinates]))
    
    return answers[0]

# Find the answer in the extracted tables
for i, data_frame in enumerate(data_frames):
    try:
        answer = get_answer(question, data_frame)
        print(f"Table {i + 1}: {answer}")
    except Exception as e:
        print(f"Error processing table {i + 1}: {e}")


KeyboardInterrupt: ignored

In [ ]:
import os
from pdf2image import convert_from_path
from PIL import Image
import camelot

pdf_file = '2.pdf'
output_folder = '/content/ex3/2'

# Convert PDF pages to images
images = convert_from_path(pdf_file)

# Loop through each image (page)
for page_num, image in enumerate(images, start=1):

    # Use camelot to read tables in the current page and store the table coordinates
    try:
        tables = camelot.read_pdf(pdf_file, flavor='stream', pages=str(page_num), edge_tol=50000)

        # If tables are found
        if tables.n:
            for i, table in enumerate(tables):
                x1, y1, x2, y2 = [int(coord) for coord in table._bbox]
                cropped_image = image.crop((x1, image.height - y2, x2, image.height - y1))

                # Save the cropped image (table region) to the output folder
                output_path = os.path.join(output_folder, f"table_page_{page_num}_table_{i + 1}.png")
                cropped_image.save(output_path)
                print(f"Saved Table {i + 1} on page {page_num} as an image.")
        else:
            print(f"No table found on page {page_num}")

    except Exception as e:
        print(f"Error processing page {page_num}: {e}")

Error processing page 1: list index out of range


In [ ]:
import PyPDF2
import pandas as pd
import tabula

# Read the PDF file
pdf_file = '1.pdf'
pdf = PyPDF2.PdfReader(open(pdf_file, 'rb'))

# Loop through each page of the PDF
for page_num in range(len(pdf.pages)):

    # Use tabula to read tables in the current page and store them as DataFrames
    try:
        tables = tabula.read_pdf(pdf_file, pages=page_num + 1, multiple_tables=True, stream=True)

        # If tables are found
        if len(tables) > 0:
            for i, table in enumerate(tables):
                print(f"Table {i + 1} on page {page_num + 1}:")
                display(table)
                print("\n")
        else:
            print(f"No table found on page {page_num + 1}")

    except Exception as e:
        print(f"Error processing page {page_num + 1}: {e}")


Error processing page 1: module 'tabula' has no attribute 'read_pdf'


In [ ]:
!pip uninstall tabula
!pip install tabula-py

Found existing installation: tabula 1.0.5
Uninstalling tabula-1.0.5:
  Would remove:
    /usr/local/lib/python3.9/dist-packages/tabula-1.0.5.dist-info/*
    /usr/local/lib/python3.9/dist-packages/tabula/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.9/dist-packages/tabula/errors/__init__.py
    /usr/local/lib/python3.9/dist-packages/tabula/file_util.py
    /usr/local/lib/python3.9/dist-packages/tabula/io.py
    /usr/local/lib/python3.9/dist-packages/tabula/py.typed
    /usr/local/lib/python3.9/dist-packages/tabula/tabula-1.0.5-jar-with-dependencies.jar
    /usr/local/lib/python3.9/dist-packages/tabula/template.py
    /usr/local/lib/python3.9/dist-packages/tabula/util.py
Proceed (Y/n)? y
  Successfully uninstalled tabula-1.0.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tabula 
df = tabula.read_pdf('my_pdf')

AttributeError: ignored

In [ ]:
from tabula import wrapper
from tabulate import tabulate
 
#reads table from pdf file
df = wrapper.read_pdf("abc.pdf",pages="all") #address of pdf file
print(tabulate(df))

ImportError: ignored

In [ ]:
!apt install ghostscript python3-tk
from ctypes.util import find_library
find_library("gs")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-tk is already the newest version (3.8.10-0ubuntu1~20.04).
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs9 libgs9-common
  libidn11 libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript libgs9
  libgs9-common libidn11 libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 10 newly installed, 0 to remove and 23 not upgraded.
Need to get 12.7 MB of archives.
After this operation, 51.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 fonts-

'libgs.so.9'

In [ ]:
!sudo pip install tabula-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import camelot
pdf_path = "1.pdf"
tables = camelot.read_pdf(pdf_path, flavor='stream', pages='all')

IndexError: ignored

In [ ]:
import camelot
file = '.pdf'
tables = camelot.read_pdf(file, pages='all', flavor='stream', row_tol=20, strip_text='\n') 
tables.export('foo.csv', f='csv', compress=False)

IndexError: ignored

In [ ]:
tables[2].df

,0,1,2,3
0,1304,,,"IEEE JOURNAL OF SOLID-STATE CIRCUITS, VOL. 52,..."
1,[21],"J. C. Pedro, L. C. Nunes,","and P. M. Cabral,\n“A simple method\nto",Peter M. Asbeck (M’75–SM’97–F’00) received the
2,,estimate the output power and efﬁciency load-p...,,B.S.\nand\nPh.D.\ndegrees\nfrom the Massachusetts
3,,"power ampliﬁers,” IEEE Trans. Microw. Theory T...",,"Institute\nof\nTechnology,\nCambridge, MA, USA,"
4,,"pp. 1239–1249, Apr. 2015.",,"in 1969 and 1975,\nrespectively."
5,,,,"He\nwas\nwith\nthe\nSarnoff\nResearch\nCenter,"
6,,,,"Princeton,\nNJ,\nUSA,\nand\nthe\nPhilips\nLabora-"
7,,,,"tory, Briarcliff Manor, NY, USA, where\nhe was"
8,,,,involved\nin\nthe\nareas\nof\nquantum\nelectro...
9,,,Voravit\nVorapipat\nreceived\nthe\nB.S.\ndegree,


# Research

In [ ]:
import cv2

image_file = 'example.pdf'
image = cv2.imread(image_file)

if image is not None:
  pass# continue with image processing
else:
  print(f"Error: Failed to load {image_file}.")

Error: Failed to load example.pdf.


In [ ]:
pdf_file = open('example3.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)
num_pages = len(pdf_reader.pages)

In [ ]:
num_pages

11

In [ ]:
df_text = pd.DataFrame(columns=['page_num', 'x', 'y', 'width', 'height', 'text'])
df_image = pd.DataFrame(columns=['page_num', 'x', 'y', 'width', 'height', 'image'])
df_table = pd.DataFrame(columns=['page_num', 'x', 'y', 'width', 'height', 'table'])
df_chart = pd.DataFrame(columns=['page_num', 'x', 'y', 'width', 'height', 'chart'])

for page_num in range(num_pages):

    print(page_num)

    page = pdf_reader.pages[page_num]
    page_media = page['/Resources']['/XObject'].get_object()
    page_image = page.mediabox

    img = page_media['/Im1']._data  # get the raw image data

    break

0


In [ ]:
page['/Resources']

{'/ColorSpace': {'/Cs6': ['/ICCBased',
   IndirectObject(127, 0, 140325689040128)],
  '/Cs8': ['/Indexed',
   IndirectObject(54, 0, 140325689040128),
   136,
   IndirectObject(167, 0, 140325689040128)]},
 '/ExtGState': {'/GS1': {'/BG2': '/Default',
   '/HT': '/Default',
   '/OP': False,
   '/OPM': 0,
   '/SA': False,
   '/SM': 0.02,
   '/Type': '/ExtGState',
   '/UCR2': '/Default',
   '/op': False}},
 '/Font': {'/F1': {'/BaseFont': '/Times-Roman',
   '/Encoding': '/MacRomanEncoding',
   '/FirstChar': 0,
   '/FontDescriptor': {'/Ascent': 699,
    '/CapHeight': 662,
    '/Descent': -217,
    '/Flags': 34,
    '/FontBBox': [-168, -218, 1000, 898],
    '/FontFile3': {'/Filter': '/FlateDecode', '/Subtype': '/Type1C'},
    '/FontName': '/Times-Roman',
    '/ItalicAngle': 0,
    '/StemH': 84,
    '/StemV': 84,
    '/Type': '/FontDescriptor',
    '/XHeight': 450},
   '/LastChar': 255,
   '/Subtype': '/Type1',
   '/Type': '/Font',
   '/Widths': [250,
    250,
    250,
    250,
    250,
    250,

In [ ]:
page_media

{'/Im1': {'/BitsPerComponent': 8,
  '/ColorSpace': ['/Indexed',
   IndirectObject(54, 0, 140325689040128),
   136,
   IndirectObject(167, 0, 140325689040128)],
  '/Filter': '/FlateDecode',
  '/Height': 798,
  '/Subtype': '/Image',
  '/Type': '/XObject',
  '/Width': 1013}}

In [ ]:
for key, value in page_media.items():
  print(key)

/Im1


In [ ]:
img_data = page_media['/Im1']._data  # get the raw image data

In [ ]:
img_file = io.BytesIO(img_data)
img_pil = Image.open(img_file)
img_pil_gray = img_pil.convert('L')  # convert to grayscale
img = np.array(img_pil_gray)  # convert to numpy array
img = img[page_image[1]:page_image[3], page_image[0]:page_image[2]]  # crop to page boundaries

UnidentifiedImageError: ignored

In [ ]:
img_file

In [ ]:
df_text = pd.DataFrame(columns=['page_num', 'x', 'y', 'width', 'height', 'text'])
df_image = pd.DataFrame(columns=['page_num', 'x', 'y', 'width', 'height', 'image'])
df_table = pd.DataFrame(columns=['page_num', 'x', 'y', 'width', 'height', 'table'])
df_chart = pd.DataFrame(columns=['page_num', 'x', 'y', 'width', 'height', 'chart'])

for page_num in range(num_pages):

    page = pdf_reader.pages[page_num]
    page_media = page['/Resources']['/XObject'].get_object()
    page_image = page.mediabox

    img = page_media['/Im1']._data  # get the raw image data



    img = cv2.imdecode(
        np.frombuffer(img, dtype='uint8'),
         cv2.IMREAD_GRAYSCALE)  # convert to grayscale image


    #img = img[page_image[1]:page_image[3], page_image[0]:page_image[2]]  # crop to page boundaries

    text = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)

    for i in range(len(text['level'])):
        if text['level'][i] == 5:
            df_text = df_text.append({
                'page_num': page_num,
                'x': text['left'][i],
                'y': text['top'][i],
                'width': text['width'][i],
                'height': text['height'][i],
                'text': text['text'][i]
            }, ignore_index=True)
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 50 and h > 50:
            if w > 3*h:
                df_image = df_image.append({
                    'page_num': page_num,
                    'x': x,
                    'y': y,
                    'width': w,
                    'height': h,
                    'image': img[y:y+h, x:x+w]
                }, ignore_index=True)
            elif h > 3*w:
                df_table = df_table.append({
                    'page_num': page_num,
                    'x': x,
                    'y': y,
                    'width': w,
                    'height': h,
                    'table': img[y:y+h, x:x+w]
                }, ignore_index=True)
            else:
                df_chart = df_chart.append({
                    'page_num': page_num,
                    'x': x,
                    'y': y,
                    'width': w,
                    'height': h,
                    'chart': img[y:y+h, x:x+w]
                }, ignore_index=True)

TypeError: ignored

In [ ]:
img

In [ ]:
import pytesseract
from PIL import Image

img = Image.open('image1.png')
text = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)
print(" ".join(text['text']))

    It was the best of  times, it was the worst  of times, it was the age  of wisdom, it was the  age of foolishness...


In [ ]:
  "12aq "